# <img src="../img/divvy_logo.svg" class="img-header"> tutorial using the command-line interface

`Divvy` also provides a command-line interface that gives you the same power as the python API. You can use `--help` to get a list of the command-line options:

In [2]:
divvy --help

version: 0.2-dev
usage: divvy [-h] [-V] [-C CONFIG] [-S SETTINGS] [-P PACKAGE] -O OUTFILE

divvy - write compute jobs that can be submitted to any computing resource

optional arguments:
  -h, --help            show this help message and exit
  -V, --version         show program's version number and exit
  -C CONFIG, --config CONFIG
                        Divvy configuration file.
  -S SETTINGS, --settings SETTINGS
                        YAML file with job settings to populate the template.
  -P PACKAGE, --package PACKAGE
                        Compute package
  -O OUTFILE, --outfile OUTFILE
                        Output filepath

https://github.com/pepkit/divvy


## The settings file

The settings argument is where you can pass an existing `yaml` file with key-value pairs. Here's a simple example:

In [7]:
cat settings.yaml

time: 4-0-0
logfile: results.log
cores: 6
partition: large_mem


Now let's take a look at the template we are going to use by activating the `slurm` package

In [8]:
cat ../divvy/submit_templates/slurm_template.sub

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='{LOGFILE}'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='{CORES}'
#SBATCH --time='{TIME}'
#SBATCH --partition='{PARTITION}'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

{CODE}


We use `divvy` to populate that template with our list of variables above, like this:

In [14]:
divvy -P slurm -S settings.yaml -O test.sub

Traceback (most recent call last):
  File "/usr/lib/python2.7/logging/__init__.py", line 861, in emit
    msg = self.format(record)
  File "/usr/lib/python2.7/logging/__init__.py", line 734, in format
    return fmt.format(record)
  File "/usr/lib/python2.7/logging/__init__.py", line 465, in format
    record.message = record.getMessage()
  File "/usr/lib/python2.7/logging/__init__.py", line 329, in getMessage
    msg = msg % self.args
TypeError: not all arguments converted during string formatting
Logged from file compute.py, line 329
No global config file was provided in environment variable(s): ['DIVCFG', 'PEPENV']
Using default config file.
Loading divvy config file: /home/nsheff/.local/lib/python2.7/site-packages/divvy/submit_templates/default_compute_settings.yaml
Available packages: default, local, slurm
Activating compute package 'default'
Activating compute package 'slurm'
Loading yaml settings file: settings.yaml
Writing script to /home/nsheff/code/divvy/docs_jupyter/test.sub

In [11]:
cat test.sub

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='results.log'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='6'
#SBATCH --time='4-0-0'
#SBATCH --partition='large_mem'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

{CODE}


We didn't include the actual command we wanted to run the `code` variable, so this template is incomplete. We can pass that variable (along with any other variables) either in the compute package, in the `settings.yaml` file, or on the command-line. The later ones have priority and will override any values in the more distant locations. For example:

In [4]:
divvy -P slurm -S settings.yaml -O test.sub --code run-this-cmd --jobname 12345 --time test

/home/nsheff/code/divvy/docs_jupyter


In [12]:
cat test.sub

#!/bin/bash
#SBATCH --job-name='{JOBNAME}'
#SBATCH --output='results.log'
#SBATCH --mem='{MEM}'
#SBATCH --cpus-per-task='6'
#SBATCH --time='4-0-0'
#SBATCH --partition='large_mem'
#SBATCH -m block
#SBATCH --ntasks=1

echo 'Compute node:' `hostname`
echo 'Start time:' `date +'%Y-%m-%d %T'`

{CODE}
